In [3]:
from matplotlib import pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score
from pandas import ExcelWriter
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import numpy as np
import re
import random


def xy_sets(line):
    X=None
    y=None
    indexes = map(int, re.findall('\d+', line))
    aux = False
    for i in indexes:
        if aux is False:
            X = scaled[i, :]
            y = target[i]
            aux = True
            continue
        X = np.vstack((X, scaled[i, :]))
        y = np.hstack((y, target[i]))
        
    return X, y, indexes


multiplier = ['0,05', '0,01', '0,001']
n_hidden = [2, 3]
alpha = [0.001, 0.01, 0.1, 1]
activation = ['identity', 'logistic', 'tanh']
dataset_name = ['57EDG', '54BND', 'combined']

for name in dataset_name:
    writer = ExcelWriter(name+'_ann.xlsx', engine='xlsxwriter')
    df = DataFrame()
    temp = DataFrame()
    for m in multiplier:
        for n in n_hidden:
            for a in alpha:
                for f in activation:
                    # load dataset
                    dataset = read_csv(name+'_'+m+'.csv', header=0, index_col=(0,1))
                    values = dataset.values

                    # split data set into data(input) and target(output)
                    data = values[:, 1:] 
                    target = values[:, 0]

                    # normalize features
                    scaled = normalize(data[:, :3], norm='max', axis=0)
                    scaled = np.concatenate((scaled, data[:, :4]), axis = 1)

                    scores = np.zeros(20)
                    col = np.array([])
                    cont = 0
                    sensitivity = 0  # true positives rate
                    specificity = 0  # true negatives rate
                    y_test_full = np.array([])
                    y_cont_full = np.array([])

                    file = open(name+'_kf.txt', 'r')
                    while cont < 20:

                        train_line = file.readline()
                        test_line = file.readline()

                        X_train, y_train, train_indexes = xy_sets(train_line)
                        X_test, y_test, test_indexes = xy_sets(test_line)
                        
                        classifier = MLPClassifier(hidden_layer_sizes=(n,), solver='lbfgs', 
                                                    alpha=a, random_state=11, verbose=True, activation=f)  # classifier 
                        classifier.fit(X_train, y_train)
                        scores[cont] = classifier.score(X_test, y_test)
                        y_pred = classifier.predict(X_test)

                        y_test_full = np.hstack((y_test_full, y_test))
                        y_cont_full = np.hstack((y_cont_full, y_pred))

                        cont += 1

                    file.close()

                    print(confusion_matrix(y_test_full, y_cont_full))

                    acc = accuracy_score(y_test_full, y_cont_full)
                    recall = recall_score(y_test_full, y_cont_full, average=None)
                    sns = recall[0]
                    spf = recall[1]
                    
                    col = np.hstack((col, a))
                    col = np.hstack((col, f))
                    col = np.hstack((col, m))
                    col = np.hstack((col, scores))
                    col = np.hstack((col, acc))
                    col = np.hstack((col, sns))
                    col = np.hstack((col, spf))
                    # col = np.hstack((col, std))

                    df.insert(0, n, col, True)
                    row_na = df.index[0]
                    df = df.rename(index={row_na: 'alpha'})
                    row_na = df.index[1]
                    df = df.rename(index={row_na: 'activation'})
                    row_na = df.index[2]
                    df = df.rename(index={row_na: 'polygonal_model'})
                    
                    row_na = df.index[-1]
                    df = df.rename(index={row_na: 'specificity'})
                    row_na = df.index[-2]
                    df = df.rename(index={row_na: 'sensibility'})
                    row_na = df.index[-3]
                    df = df.rename(index={row_na: 'accuracy'})
                    
                    print(scores.mean())
                    print("Accuracy: %0.2f" % (acc))
                    print("Sensitivity: %0.2f" % (sns))
                    print("Specificity: %0.2f" % (spf))

    df.to_excel(writer, sheet_name=name)
    writer.save()
    writer.close()
    

[[34  3]
 [12  8]]
0.7416666666666667
Accuracy: 0.74
Sensitivity: 0.92
Specificity: 0.40
[[32  5]
 [11  9]]
0.725
Accuracy: 0.72
Sensitivity: 0.86
Specificity: 0.45
[[34  3]
 [12  8]]
0.7416666666666667
Accuracy: 0.74
Sensitivity: 0.92
Specificity: 0.40
[[34  3]
 [12  8]]
0.7416666666666667
Accuracy: 0.74
Sensitivity: 0.92
Specificity: 0.40
[[33  4]
 [ 9 11]]
0.7833333333333333
Accuracy: 0.77
Sensitivity: 0.89
Specificity: 0.55
[[32  5]
 [10 10]]
0.75
Accuracy: 0.74
Sensitivity: 0.86
Specificity: 0.50
[[34  3]
 [12  8]]
0.7416666666666667
Accuracy: 0.74
Sensitivity: 0.92
Specificity: 0.40
[[35  2]
 [12  8]]
0.7583333333333333
Accuracy: 0.75
Sensitivity: 0.95
Specificity: 0.40
[[35  2]
 [14  6]]
0.725
Accuracy: 0.72
Sensitivity: 0.95
Specificity: 0.30
[[35  2]
 [12  8]]
0.7583333333333333
Accuracy: 0.75
Sensitivity: 0.95
Specificity: 0.40
[[37  0]
 [20  0]]
0.65
Accuracy: 0.65
Sensitivity: 1.00
Specificity: 0.00
[[35  2]
 [12  8]]
0.7583333333333333
Accuracy: 0.75
Sensitivity: 0.95
Spec

[[18 10]
 [24  2]]
0.35
Accuracy: 0.37
Sensitivity: 0.64
Specificity: 0.08
[[18 10]
 [ 9 17]]
0.6333333333333333
Accuracy: 0.65
Sensitivity: 0.64
Specificity: 0.65
[[21  7]
 [ 6 20]]
0.7583333333333333
Accuracy: 0.76
Sensitivity: 0.75
Specificity: 0.77
[[20  8]
 [ 4 22]]
0.7833333333333333
Accuracy: 0.78
Sensitivity: 0.71
Specificity: 0.85
[[19  9]
 [ 9 17]]
0.6583333333333333
Accuracy: 0.67
Sensitivity: 0.68
Specificity: 0.65
[[19  9]
 [ 4 22]]
0.7583333333333333
Accuracy: 0.76
Sensitivity: 0.68
Specificity: 0.85
[[21  7]
 [ 5 21]]
0.775
Accuracy: 0.78
Sensitivity: 0.75
Specificity: 0.81
[[19  9]
 [ 9 17]]
0.6583333333333333
Accuracy: 0.67
Sensitivity: 0.68
Specificity: 0.65
[[17 11]
 [ 5 21]]
0.6916666666666667
Accuracy: 0.70
Sensitivity: 0.61
Specificity: 0.81
[[20  8]
 [ 4 22]]
0.775
Accuracy: 0.78
Sensitivity: 0.71
Specificity: 0.85
[[17 11]
 [ 6 20]]
0.6666666666666666
Accuracy: 0.69
Sensitivity: 0.61
Specificity: 0.77
[[17 11]
 [ 7 19]]
0.65
Accuracy: 0.67
Sensitivity: 0.61
Spec

[[54 11]
 [10 36]]
0.8116666666666668
Accuracy: 0.81
Sensitivity: 0.83
Specificity: 0.78
[[52 13]
 [ 8 38]]
0.8116666666666668
Accuracy: 0.81
Sensitivity: 0.80
Specificity: 0.83
[[53 12]
 [ 9 37]]
0.8133333333333335
Accuracy: 0.81
Sensitivity: 0.82
Specificity: 0.80
[[53 12]
 [ 6 40]]
0.8416666666666668
Accuracy: 0.84
Sensitivity: 0.82
Specificity: 0.87
[[48 17]
 [ 8 38]]
0.7766666666666667
Accuracy: 0.77
Sensitivity: 0.74
Specificity: 0.83
[[53 12]
 [ 9 37]]
0.8133333333333335
Accuracy: 0.81
Sensitivity: 0.82
Specificity: 0.80
[[53 12]
 [ 9 37]]
0.8133333333333335
Accuracy: 0.81
Sensitivity: 0.82
Specificity: 0.80
[[51 14]
 [ 9 37]]
0.7983333333333335
Accuracy: 0.79
Sensitivity: 0.78
Specificity: 0.80
[[53 12]
 [ 9 37]]
0.8133333333333335
Accuracy: 0.81
Sensitivity: 0.82
Specificity: 0.80
[[51 14]
 [ 6 40]]
0.8233333333333335
Accuracy: 0.82
Sensitivity: 0.78
Specificity: 0.87
[[52 13]
 [ 8 38]]
0.8133333333333335
Accuracy: 0.81
Sensitivity: 0.80
Specificity: 0.83
[[54 11]
 [ 9 37]]
0.